In [1]:
#| default_exp clean



A lot of the data we captured is in the form of lists of items. In order to search these, we need to pull them out into separate tables.

In [11]:
# TODO: Retreive this from the key model
array_items = "more_company_info","job_requirements","job_soft_skills","tech_stack","information_urls"

Back up the data before we start

In [3]:
# from hn_jobs_chat.db import connectToDB
# from hn_jobs_chat.vars import postsTableName

# newTableName = postsTableName + "_tech_stack"

# # Connect to the database
# conn, cursor = connectToDB()

# # Create a new table for the array data
# query = """CREATE TABLE IF NOT EXISTS """ + newTableName + """ (
#         id SERIAL PRIMARY KEY, 
#         postId INT, 
#         tech TEXT
#         );"""

# cursor.execute(query)
# conn.commit()   

# # Retrieve all the data from the primary table
# query = """SELECT id, tech_stack FROM """ +  postsTableName +  """ ORDER BY id"""

# cursor.execute(query)
# items = cursor.fetchall()

# # Iterate over the items, 
# for item in items:
#     id, tech_stack = item
#     if tech_stack is not None:
#         if tech_stack != "":
#             tech_stack_items = tech_stack.split(",")

#             for tech in tech_stack_items:
#                 query = """INSERT INTO """+newTableName+""" (postId, tech) VALUES (%s, %s)"""
#                 cursor.execute(query, (id,  tech))

# conn.commit()


Ok, now we have need to convert the array into a new table, so we can search against the embeddings.

In [12]:
from hn_jobs_chat.db import connectToDB

postsTableName = "posts_august"
# Iterate over all array items

for array_item in array_items:
    newTableName = f"{postsTableName}_{array_item}"
    
    # Connect to the database
    conn, cursor = connectToDB()

    # Create a new table for the array data
    query = f"""CREATE TABLE IF NOT EXISTS {newTableName} (
            id SERIAL PRIMARY KEY, 
            postId INT, 
            item TEXT,
            embedding float[1536]
            );"""

    cursor.execute(query)
    conn.commit()   

    # Retrieve all the data from the primary table
    query = f"""SELECT id, {array_item} FROM {postsTableName} ORDER BY id"""

    cursor.execute(query)
    items = cursor.fetchall()

    # Iterate over the items
    for item in items:
        id, array_data = item
        if array_data is not None and array_data != "":
            array_items = array_data.split(",")

            for sub_item in array_items:
                query = f"""INSERT INTO {newTableName} (postId, item) VALUES (%s, %s)"""
                cursor.execute(query, (id, sub_item.strip()))

    conn.commit()
    conn.close()


In [ ]:
from hn_jobs_chat.db import connectToDB
